<a href="https://colab.research.google.com/github/Fernando-LunaP/Ing-Caracteristicas/blob/main/Proyecto_1/Proyecto1_Descargando_los_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Ingeniería de Características</h1>

<h3>Proyecto 1: Descargando los datos</h3>

<p> Alumno </p>
<p> Fernando Luna Ponce </p>

</center>

En este proyecto realizaremos la obtención de datos sobre la diabetes, en específico para el Estado de Sonora, se estarán descargando los datos abiertos de la Dirección General de Información en Salud y de la página Cuéntame de México administrada por el INEGI.

Se quiere analizar analizar la distribución de las defunciones por diabetes en el Estado de Sonora, pero hay pocas fuentes en las que se puede encontrar información. Se encontraron dos fuentes que son:  
1. ~~Página de la [DGIS](http://www.dgis.salud.gob.mx/contenidos/basesdedatos/da_defunciones_gobmx.html): desde esta página se puede descargar la información de las muertes para diferentes años, esta viene en formato zip, por lo que hay que realizar una descompresión para obtener el archivo que nos interesa en formato csv.~~ En revisión posterior no se pudo realizar la conexión con la página por lo que se descargará de otra dirección el mismo dataset. [Aquí](https://www.inegi.org.mx/contenidos/programas/mortalidad/datosabiertos/defunciones/2020/conjunto_de_datos_defunciones_registradas_2020_csv.zip)

2. [Cuéntame de México](https://cuentame.inegi.org.mx/monografias/informacion/son/poblacion/default.aspx?tema=me&e=26): esta página contiene la información que nos interesa sobre el número de habitantes por municipio en Sonora en el 2020, si bien se podrían encontrar estos datos en otro lugar y con un formato más manejable, se decidió usar la información de esta página  para mostrar además como es el método de web scraping para extraer los datos de una tabla en formato html y almacenar estos en un dataframe.

1. Antes de empezar vamos a cargar la librerías necesarias para la descarga y limpieza de los datos

In [151]:
import os  # Para manejo de archivos y directorios
import urllib.request # Una forma estandard de descargar datos
# import requests # Otra forma no de las librerías de uso comun

import datetime # Fecha de descarga
import pandas as pd # Solo para ver el archivo descargado
import zipfile # Descompresión de archivos
import re # Expresiones regulares
import csv # Para abrir csv
import zipfile # Descompresión de archivos

from bs4 import BeautifulSoup
import requests

import numpy as np

A continuación vamos a crear algunas funciones que nos ayudarán a mantener un código más legible además de simplificar las tareas en proyectos posteriores

In [152]:
def descargadatos(url, archivo, subdir): #Para descargar un archivo
  if not os.path.exists(archivo):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    # Empezamos a descargar el primer archivo
    urllib.request.urlretrieve(url, subdir + archivo)

def descomprimeZip(subdir, archivo): # Descomprime el csv y le cambia el nombre
    with zipfile.ZipFile(subdir + archivo, "r") as zip_ref:
        nombre = zip_ref.namelist()
        zip_ref.extractall(subdir)

# Descargando los datos

**Descarga datos fuente 1:**

Descargamos el zip y le cambiamos el nombre, luego extraemos las carpetas que contiene y le cambiamos el nombre al csv que nos interesa.

In [153]:
url_inegi = "https://www.inegi.org.mx/contenidos/programas/mortalidad/datosabiertos/defunciones/2020/conjunto_de_datos_defunciones_registradas_2020_csv.zip"
archivoZIP = "Defunciones_" + str(2020) + ".zip"
archivoCSV = "Defunciones_" + str(2020) + ".csv"
subdir = "./data/fuente_1/"
descargadatos(url_inegi, archivoZIP, subdir)
descomprimeZip(subdir, archivoZIP)
os.rename(subdir + 'conjunto_de_datos/conjunto_de_datos_defunciones_registrados_2020.csv', subdir + archivoCSV)

Leemos el contenido

In [154]:
rawDataDefunciones =  pd.read_csv(subdir+archivoCSV)
rawDataDefunciones.head()

,ent_regis,mun_regis,ent_resid,mun_resid,tloc_resid,loc_resid,ent_ocurr,mun_ocurr,tloc_ocurr,loc_ocurr,...,anio_cert,maternas,lengua,cond_act,par_agre,ent_ocules,mun_ocules,loc_ocules,razon_m,dis_re_oax
0,1,1,1,1,15,1,1,1,15,1,...,2020,NaN,2,9,88,88,888,8888,0,999
1,1,1,1,1,15,1,1,1,15,1,...,2020,NaN,9,9,88,88,888,8888,0,999
2,1,1,1,1,15,1,1,1,15,1,...,2020,NaN,2,2,88,88,888,8888,0,999
3,1,1,1,1,15,1,1,1,15,1,...,2020,NaN,9,9,88,88,888,8888,0,999
4,1,6,1,9,4,1,1,9,4,1,...,2020,NaN,2,1,88,88,888,8888,0,999


**Descarga datos fuente 2:**

Nos interesó una tabla que contiene la información del número de habitantes en Sonora, pero no hay un archivo descargable, por lo que tenemos que hacer web scraping para poder descargar esa información.

In [155]:
url_cuentame = "https://cuentame.inegi.org.mx/monografias/informacion/son/poblacion/default.aspx?tema=me&e=26"
page = requests.get(url_cuentame) # Realizamos la petición a la url solicitada
soup = BeautifulSoup(page.content, 'html.parser') # Le damos formato html a la información

tabla = soup.find('table', id="keywords2")

In [156]:
# Generamos listas para almacenar toda la información que deseamos almacenar
listaCdMun = list()
listaNbMun = list()
listaCantidad = list()

In [157]:
filas = tabla.findAll('tr')
for i in range(len(filas)):
  if i > 0:
    columnas = filas[i].findAll('td')
    cd_municipio = columnas[0].string
    listaCdMun.append(cd_municipio)
    nb_municipio = columnas[1].string.replace('    ', ' ')
    listaNbMun.append(nb_municipio)
    nu_habitantes = columnas[2].string.replace(',', '')
    listaCantidad.append(nu_habitantes)

Generamos nuestro dataframe con toda la información que obtuvimos

In [158]:
rawDataHabitantes = pd.DataFrame({"Codigo_Municipio":listaCdMun,
                         "Nombre_Municipio":listaNbMun,
                         "Cantidad_Habitantes":listaCantidad})
rawDataHabitantes.head()

,Codigo_Municipio,Nombre_Municipio,Cantidad_Habitantes
0,001,Aconchi,2563
1,002,Agua Prieta,91929
2,003,Alamos,24976
3,004,Altar,9492
4,005,Arivechi,1177


**Documentando el origen de nuestros datos**

In [159]:
# Generamos el documento para indicar los datos generales de la descarga
subdir = "./data/"
with open(subdir + "info.txt", 'w') as f:
      f.write("Archivo con información referente a la población y diabetes\n")
      info = """
      Datos nacionales de defunciones

      Los datos se obtuvieron de la página del INEGI con la información correspondiente
      a las defunciones nacionales registradas del año 2020

      """ 
      f.write(info + '\n')
      f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
      f.write("Desde: " + url_inegi + "\n")
      f.write("Nombre: " + archivoCSV + "\n")

      info = """
      Datos de habitantes por municipio Sonora

      Los datos se obtuvieron de la página Cuéntame de México con la información del
      número de habitantes por municipio del Estado de Sonora del año 2020

      """

      f.write(info + '\n')
      f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
      f.write("Desde: " + url_cuentame + "\n")

# Limpieza para generar datos tidy

**Fuente 1**

Según la vista inicial al dataframe vemos que se tienen una gran cantidad de columnas pero no necesitaremos todas, por lo que primero tenemos que entender cuales vamos a quitar, para esto nos basaremos en nuestro en el diccionario de datos que viene junto con nuestros datos.

In [160]:
subdir = "./data/fuente_1/"
RutaDiccionario = subdir + 'diccionario_de_datos/diccionario_datos_defunciones_registradas_2020.csv'
dicRawDefunciones = pd.read_csv(RutaDiccionario, encoding='latin-1')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(dicRawDefunciones)

                                         NOMBRE_CAMPO  LONGITUD TIPO  \
0                                 Entidad de registro         2    C   
1                    Municipio o alcaldía de registro         3    C   
2   Entidad de residencia habitual del (la) fallec...         2    C   
3   Municipio o alcaldía de residencia habitual de...         3    C   
4   Tamaño de localidad de residencia habitual del...         2    N   
5   Localidad de residencia habitual del (la) fall...         4    C   
6                               Entidad de ocurrencia         2    C   
7                  Municipio o alcaldía de ocurrencia         3    C   
8                   Tamaño de localidad de ocurrencia         2    N   
9                             Localidad de ocurrencia         4    C   
10            Causa de la defunción (lista detallada)         4    C   
11             Causa de la defunción (lista mexicana)         3    C   
12                        Sexo del (la) fallecido (a)         1 

Después de observar la composición de este dataframe vemos que tenemos que seguir los siguientes pasos para obtener la información que nos interesa:

1. Filtrar por Estado de residencia habitual del fallecido, ya que este campo es el que nos indica donde radicaba, en este caso solo nos interesan las correspondientes al Estado de Sonora (26)

In [161]:
rawDataDefunciones = rawDataDefunciones[rawDataDefunciones["ent_resid"] == 26]
rawDataDefunciones.head()

,ent_regis,mun_regis,ent_resid,mun_resid,tloc_resid,loc_resid,ent_ocurr,mun_ocurr,tloc_ocurr,loc_ocurr,...,anio_cert,maternas,lengua,cond_act,par_agre,ent_ocules,mun_ocules,loc_ocules,razon_m,dis_re_oax
10051,2,2,26,55,13,1,2,2,15,1,...,2020,NaN,2,1,88,88,888,8888,0,999
10131,2,2,26,55,13,1,2,2,15,1,...,2020,NaN,8,8,88,88,888,8888,0,999
10171,2,2,26,55,13,1,2,2,15,1,...,2020,NaN,2,2,88,88,888,8888,0,999
10207,2,2,26,55,13,1,2,2,15,1,...,2020,NaN,2,2,88,88,888,8888,0,999
10209,2,2,26,55,13,1,2,2,15,1,...,2020,NaN,2,2,88,88,888,8888,0,999


2. Filtrar solo los que tengan causa de defunción por diabetes, para esto tenemos abrimos otro excel con la lista de nuestros valores para la columna de causa de defunción. Vamos a extraer estos valores y con estos filtraremos nuestro dataframe.

In [162]:
causaDef = pd.read_csv('/content/data/fuente_1/catalogos/causa_defuncion.CSV', encoding='latin-1')
causaDiabetes = causaDef.query('descrip.str.contains("Diabetes")', engine='python')
causaDiabetes.head()

,cve,descrip
873,E100,"Diabetes mellitus insulinodependiente, con coma"
874,E101,"Diabetes mellitus insulinodependiente, con cet..."
875,E102,"Diabetes mellitus insulinodependiente, con com..."
876,E104,"Diabetes mellitus insulinodependiente, con com..."
877,E105,"Diabetes mellitus insulinodependiente, con com..."


In [163]:
rawDataDefunciones = rawDataDefunciones[rawDataDefunciones['causa_def'].isin(causaDiabetes["cve"])]

3. Vamos a dejar las columnas ent_resid, mun_resid, sexo, edad, asis_medi

In [164]:
rawDataDefunciones = rawDataDefunciones[["mun_resid", "sexo", "edad", "asist_medi"]]
rawDataDefunciones.head()

,mun_resid,sexo,edad,asist_medi
12975,48,1,4075,1
13311,55,2,4052,1
18876,55,2,4079,9
19528,42,1,4072,1
21614,70,2,4068,9


4. El siguiente paso es filtrar los valores que si estén especificados de sexo, edad y asis_medi.

In [165]:
rawDataDefunciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2268 entries, 12975 to 996981
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   mun_resid   2268 non-null   int64
 1   sexo        2268 non-null   int64
 2   edad        2268 non-null   int64
 3   asist_medi  2268 non-null   int64
dtypes: int64(4)
memory usage: 88.6 KB


5. Con las columnas de sexo y asis_medi a generar nuevos dataframes, ya que debemos generar una columna nueva por cada uno de los valores posibles de edad y asist_medi

Empezamos con la columna de sexo, vamos a separarla en dos columnas, si es hombre o mujer y después generarmos un nuevo dataframe con las funciones de agregación para obtener el total de hombres y mujeres agrupados por municipio de residencia.

In [166]:
valores = [1, 2]
vSexo = rawDataDefunciones[rawDataDefunciones["sexo"].isin(valores)].iloc[:, [0, 1]]
vSexo["esHombre"] = np.where(vSexo['sexo'] == 1, 1, 0)
vSexo["esMujer"] = np.where(vSexo['sexo'] == 2, 1, 0)
vSexo.drop(columns=["sexo"])
df_sexo = vSexo.groupby('mun_resid').agg(cant_Hombres=('esHombre', 'sum'),
                       cant_Mujeres=('esMujer', 'sum'))
df_sexo.head()

,cant_Hombres,cant_Mujeres
mun_resid,,
1,0,2
2,39,53
3,6,7
4,5,6
6,2,4


Seguimos con la columna de asist_medi, pero en este caso solo nos interesa generar una columna de los que tuvieron asistencia medica.
Nota: Previamente se habían filtrado los valores de los que tuvieron o no asistencia médica especificada.

In [167]:
valores = [1, 2]
vAsist = rawDataDefunciones[rawDataDefunciones["asist_medi"].isin(valores)].iloc[:, [0, 3]]
vAsist["conAsist"] = np.where(vAsist['asist_medi'] == 1, 1, 0)
vAsist.drop(columns=["asist_medi"])
df_Asist = vAsist.groupby('mun_resid').agg(cant_Asist=('conAsist', 'sum'))
df_Asist.head()

,cant_Asist
mun_resid,
1,2
2,91
3,9
4,7
6,2


6. Para la columna de edad, los valores tienen sus propios código definidos en un catálogo que viene en conjunto con los datos descargados, como el valor es numérico podemos aplicarle una transformación a esos datos para obtener el valor en años de la edad, también generamos un dataframe con la edad promedio de las muertes por municipio.

In [168]:
valores = [4998] # Valor no especificado
vEdad = rawDataDefunciones[-rawDataDefunciones["edad"].isin(valores)].iloc[:, [0, 2]] # Agregamos el simbolo "-" para filtrar los valores que no estén en el df
vEdad["Edad"] = vEdad['edad'].apply(lambda x: 1 if x <= 4001 else x - 4000)
vEdad.drop(columns=["edad"])
df_Edad = vEdad.groupby('mun_resid').agg(EdadPromedio=('Edad', 'mean'))
df_Edad.head()

,EdadPromedio
mun_resid,
1,58.500000
2,67.467391
3,68.923077
4,67.909091
6,78.500000


**Fuente 2**

En este caso no se cuenta con un diccionario de datos y al guardar todos los valores directo de la tabla html, el tipo de datos de las columnas debería ser object, por lo que hay que transformarlo al tipo de datos que ocupemos.

1. Revisamos el tipo de datos y los valores nulos

In [169]:
rawDataHabitantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Codigo_Municipio     72 non-null     object
 1   Nombre_Municipio     72 non-null     object
 2   Cantidad_Habitantes  72 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB


2. Realizamos la conversión de las columnas, en este caso solo transformamos las columnas Codigo_Municipio y Cantidad_Habitantes a tipo numérico.

In [170]:
rawDataHabitantes["Codigo_Municipio"] = pd.to_numeric(rawDataHabitantes["Codigo_Municipio"])
rawDataHabitantes["Cantidad_Habitantes"] = pd.to_numeric(rawDataHabitantes["Cantidad_Habitantes"])
rawDataHabitantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Codigo_Municipio     72 non-null     int64 
 1   Nombre_Municipio     72 non-null     object
 2   Cantidad_Habitantes  72 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ KB


# Generación de dataframe con datos tidy

Después de todo este proceso obtuvimos varios dataframes, y al final todos se unirán para crear uno solo, por lo que ocupamos hacer múltiples join para generarlo.

1. Unir toda la información de sexo, edad y asistencia médica

In [171]:
DatosDefunciones = df_sexo.merge(df_Asist,on='mun_resid').merge(df_Edad,on='mun_resid')
DatosDefunciones.shape

(56, 4)

Tenemos un dataframe con 56 filas, lo que significa que no tenemos información de todos los municipios, porque probablemente no hay defunciones registradas para esos municipios.

2. Finalmente unimos los datos de las defunciones con los datos de habitantes por municipio para formar una sola tabla con los datos que nos interesa conocer.

In [172]:
Tidy_Data = pd.merge(rawDataHabitantes, DatosDefunciones, how='left', left_on='Codigo_Municipio', right_on='mun_resid')

Rellenamos con 0 los valores faltantes transformamos algunas columnas a tipo entero.

In [173]:
Tidy_Data = Tidy_Data.fillna(0)
Tidy_Data['cant_Hombres'] = Tidy_Data['cant_Hombres'].astype('int')
Tidy_Data['cant_Mujeres'] = Tidy_Data['cant_Mujeres'].astype('int')
Tidy_Data['cant_Asist'] = Tidy_Data['cant_Asist'].astype('int')
Tidy_Data

,Codigo_Municipio,Nombre_Municipio,Cantidad_Habitantes,cant_Hombres,cant_Mujeres,cant_Asist,EdadPromedio
0,1,Aconchi,2563,0,2,2,58.500000
1,2,Agua Prieta,91929,39,53,91,67.467391
2,3,Alamos,24976,6,7,9,68.923077
3,4,Altar,9492,5,6,7,67.909091
4,5,Arivechi,1177,0,0,0,0.000000
...,...,...,...,...,...,...,...
67,68,Villa Pesqueira,1043,1,0,1,78.000000
68,69,Yécora,4793,1,1,2,62.000000
69,70,General Plutarco Elías Calles,13627,8,10,6,66.722222
70,71,Benito Juárez,21692,12,10,20,65.818182


In [174]:
Tidy_Data.dtypes

Codigo_Municipio         int64
Nombre_Municipio        object
Cantidad_Habitantes      int64
cant_Hombres             int64
cant_Mujeres             int64
cant_Asist               int64
EdadPromedio           float64
dtype: object

# Generación de diccionario de datos para la tabla tidy

In [175]:
lista = [[] for _ in range(len(Tidy_Data.columns))]
lista[0] = ["Codigo_Municipio", "Código INEGI de municipio de Sonora", "Numérico (int64)", "Fuente 2"]
lista[1] = ["Nombre_Municipio", "Nombre del municipio", "String (object)", "Fuente 2"]
lista[2] = ["Cantidad_Habitantes", "Cantidad de habitantes del municipio", "Numérico (int64)", "Fuente 2"]
lista[3] = ["cant_Hombres", "Cantidad de defunciones masculinas por municipio", "Numérico (int64)", "Fuente 1: derivado columna sexo (suma)"]
lista[4] = ["cant_Mujeres", "Cantidad de defunciones femeninas por municipio", "Numérico (int64)", "Fuente 1: derivado columna sexo (suma)"]
lista[5] = ["cant_Asist", "Cantidad de defunciones con asistencia médica", "Numérico (int64)", "Fuente 1: suma"]
lista[6] = ["EdadPromedio", "Edad promedio de las defunciones por municipio", "Numérico (float64)", "Fuente 1: promedio"]
diccionario = pd.DataFrame(lista, columns=["Nombre_Variable", "Descripción","Tipo_Dato", "Detalles_Obtención"])
diccionario

,Nombre_Variable,Descripción,Tipo_Dato,Detalles_Obtención
0,Codigo_Municipio,Código INEGI de municipio de Sonora,Numérico (int64),Fuente 2
1,Nombre_Municipio,Nombre del municipio,String (object),Fuente 2
2,Cantidad_Habitantes,Cantidad de habitantes del municipio,Numérico (int64),Fuente 2
3,cant_Hombres,Cantidad de defunciones masculinas por municipio,Numérico (int64),Fuente 1: derivado columna sexo (suma)
4,cant_Mujeres,Cantidad de defunciones femeninas por municipio,Numérico (int64),Fuente 1: derivado columna sexo (suma)
5,cant_Asist,Cantidad de defunciones con asistencia médica,Numérico (int64),Fuente 1: suma
6,EdadPromedio,Edad promedio de las defunciones por municipio,Numérico (float64),Fuente 1: promedio


Guardamos nuestro dataframe con datos tidy y nuestro diccionario de datos

In [176]:
subdir = "./data/tidy_data/"
if not os.path.exists(subdir):
        os.makedirs(subdir)
Tidy_Data.to_parquet(subdir + 'Tidy_data.parquet')
diccionario.to_parquet(subdir + 'Diccionario.parquet')